In [1]:
import pickle
import glob
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import log_loss
from keras.layers import *
from keras import optimizers
from keras import backend as K
from sklearn.metrics import f1_score
# import time
# print('sleeping')
# time.sleep(7200)
# print('sleep done =======================')
# load feats
all = True
if all:
    train_x,test_x = [],[]
    for feat in sorted(glob.glob('../feature/stacking_pkl_file/*')):
        if 'fm2.pkl' in feat or 'ligbm' in  feat :
            continue
        print('file path',feat)
        a,b = pickle.load(open(feat,'rb'))
        print(a.shape,b.shape)
        train_x.append(a)
        test_x.append(b)
else:
    train_x,test_x = [],[]
    for feat in sorted(glob.glob('../feature/staclomg_pkl_file_nn/*')):
        if 'fm2.pkl' in feat or 'ligbm' in feat :
            continue
        print('file path',feat)
        a,b = pickle.load(open(feat,'rb'))
        print(a.shape,b.shape)
        train_x.append(a)
        test_x.append(b)

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


file path ../feature/stacking_pkl_file/Capsule.pkl
(102277, 19) (102277, 19)
file path ../feature/stacking_pkl_file/capsule_all_shuffle.pkl
(102277, 19) (102277, 19)
file path ../feature/stacking_pkl_file/capsule_shuffle.pkl
(102277, 19) (102277, 19)
file path ../feature/stacking_pkl_file/cnn.pkl
(102277, 19) (102277, 19)
file path ../feature/stacking_pkl_file/mlp.pkl
(102277, 19) (102277, 19)
file path ../feature/stacking_pkl_file/mlp_shuffle.pkl
(102277, 19) (102277, 19)
file path ../feature/stacking_pkl_file/pseudo_rnn2.pkl
(102277, 19) (102277, 19)
file path ../feature/stacking_pkl_file/rnn7879.pkl
(102277, 19) (102277, 19)
file path ../feature/stacking_pkl_file/rnn_all_shuffle_all.pkl
(102277, 19) (102277, 19)
file path ../feature/stacking_pkl_file/rnn_cnn.pkl
(102277, 19) (102277, 19)
file path ../feature/stacking_pkl_file/rnn_pseudo1.pkl
(102277, 19) (102277, 19)
file path ../feature/stacking_pkl_file/rnn_shuffle1.pkl
(102277, 19) (102277, 19)
file path ../feature/stacking_pkl_f

In [2]:
for feat in sorted(glob.glob('../feature/other_features/features-vinson/*')):
    if 'fm2.pkl' in feat or 'w2v' in feat or 'lda' in feat or 'lsi' in feat:
        continue
    print('file path',feat)
    file=pd.read_csv(feat)
    a = file.iloc[:102277,:]
    b = file.iloc[102277:,:]
    a = a.values
    b = b.values
    print(a.shape,b.shape)
    train_x.append(a)
    test_x.append(b)
train_x = np.nan_to_num(np.hstack(train_x))
test_x = np.nan_to_num(np.hstack(test_x))
print(train_x.shape)

file path ../feature/other_features/features-vinson/bnb_prob_10w.csv
(102277, 19) (102277, 19)
file path ../feature/other_features/features-vinson/lr_prob.csv
(102277, 19) (102277, 19)
file path ../feature/other_features/features-vinson/lr_prob_0.778620.csv
(102277, 19) (102277, 19)
file path ../feature/other_features/features-vinson/mnb_prob_10w.csv
(102277, 19) (102277, 19)
file path ../feature/other_features/features-vinson/nn_ensemble_0.775597.csv
(102277, 19) (102277, 19)
file path ../feature/other_features/features-vinson/svc_prob.csv
(102277, 19) (102277, 19)
file path ../feature/other_features/features-vinson/svc_prob_0.778881.csv
(102277, 19) (102277, 19)
file path ../feature/other_features/features-vinson/word_fasttext_0.760225.csv
(102277, 19) (102277, 19)
(102277, 399)


In [3]:
train = pd.read_csv("../input/new_data/train_set.csv")
y=(train["class"]-1).astype(int)
from keras.utils import np_utils
train_y=np_utils.to_categorical(y,num_classes=20)

/usr/local/lib/python3.5/dist-packages/pandas/core/computation/check.py:17: UserWarning: The installed version of numexpr 2.4.3 is not supported in pandas and will be not be used
The minimum supported version is 2.4.6

  ver=ver, min_ver=_MIN_NUMEXPR_VERSION), UserWarning)


In [4]:
def self_f1_score(data, y_hat):
    y_true = data
    y_hat = np.round(y_hat) # scikits f1 doesn't like probabilities
    return f1_score(y_true, y_hat, average='macro')

In [5]:
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.metrics import roc_auc_score
def simple_ens(model_name,k=3,rnd=233,lr=0.05,feature_fraction=0.9,bagging_fraction=0.9,
               bag_frec=3,met='binary_logloss'):
    kf = StratifiedKFold(n_splits=k, shuffle=True, random_state=rnd)
    test_pred, train_pred = np.zeros((102277,19)), np.zeros((102277,19))
    now_nfold=0
    for train_index, test_index in kf.split(train_x,y):
        
        now_nfold+=1
        print ("now is {} fold".format(now_nfold))
        curr_x,curr_y = train_x[train_index],y[train_index]
        hold_out_x,hold_out_y = train_x[test_index],y[test_index]
        d_test = test_x
        
        model = LogisticRegression(C=4, dual=True)
        model.fit(curr_x, curr_y)
        vaild = model.predict_proba(hold_out_x)
        train_pred[test_index] = vaild
        val_result = np.argmax(vaild,axis=1)
        print ('score is {}'.format(f1_score(hold_out_y, val_result, average='macro')))
        curr_test_pred = model.predict_proba(d_test)
        test_pred += curr_test_pred

    test_pred = test_pred / 10
    print('-------------------------------')
    return train_pred, test_pred

print('done')

done


In [6]:
train_pred, lr_res = simple_ens('lgb',10,233,0.05,0.6)

now is 1 fold
score is 0.810170429683372
now is 2 fold
score is 0.8132957801098499
now is 3 fold
score is 0.8039535290427603
now is 4 fold
score is 0.8087108449701125
now is 5 fold
score is 0.8080616541348702
now is 6 fold
score is 0.8041241628629686
now is 7 fold
score is 0.8060599294824313
now is 8 fold
score is 0.8073683359392362
now is 9 fold
score is 0.8089595744392909
now is 10 fold
score is 0.8070551598210907
-------------------------------


In [7]:
with open('../feature/2leve_lr_stacking2.pkl','wb') as fout:
    pickle.dump([train_pred,lr_res],fout)

In [8]:
name = ["class_prob_%s"%i for i in range(1,20)]
df_lr = pd.DataFrame(lr_res, columns=name)
# df_lr = df_lr.drop('class_prob_20', axis=1)
df_lr.to_csv('../pro/stacking_lr6.csv', index=False)

In [9]:
df_lr

,class_prob_1,class_prob_2,class_prob_3,class_prob_4,class_prob_5,class_prob_6,class_prob_7,class_prob_8,class_prob_9,class_prob_10,class_prob_11,class_prob_12,class_prob_13,class_prob_14,class_prob_15,class_prob_16,class_prob_17,class_prob_18,class_prob_19
0,0.005173,0.001242,0.001032,0.001691,0.964516,0.002928,0.000493,0.003843,0.000179,0.001428,0.002243,0.004789,0.002524,0.001960,0.001719,6.204818e-04,0.000203,0.001023,0.002393
1,0.016229,0.002644,0.000897,0.946115,0.000970,0.001312,0.005503,0.000479,0.000187,0.003628,0.001470,0.007096,0.001500,0.002156,0.000363,3.911326e-04,0.000397,0.007323,0.001340
2,0.011988,0.003675,0.017519,0.010706,0.007202,0.001943,0.013303,0.075739,0.010768,0.002357,0.016445,0.046814,0.746262,0.003589,0.011556,2.756444e-07,0.006819,0.002100,0.011215
3,0.001659,0.000224,0.000452,0.989424,0.000226,0.001856,0.000903,0.000839,0.000099,0.000565,0.001051,0.000648,0.000385,0.000240,0.000063,3.424475e-05,0.000213,0.000300,0.000819
4,0.039349,0.000614,0.002532,0.001914,0.891316,0.002622,0.005959,0.005118,0.000479,0.015960,0.007884,0.003070,0.005921,0.007824,0.000455,1.098197e-04,0.000244,0.000808,0.007819
5,0.021328,0.003874,0.006070,0.003716,0.855888,0.009398,0.000354,0.022570,0.001489,0.036153,0.010370,0.010138,0.006146,0.003271,0.000602,9.111239e-04,0.000745,0.002724,0.004253
6,0.009791,0.000482,0.005391,0.001020,0.000047,0.001954,0.000683,0.003830,0.000214,0.018266,0.000512,0.003756,0.000790,0.000717,0.933986,1.668871e-03,0.003638,0.001541,0.011714
7,0.029421,0.046175,0.007601,0.005387,0.004315,0.009586,0.037114,0.011903,0.008516,0.010668,0.012516,0.020867,0.030894,0.005865,0.006819,3.475467e-04,0.084949,0.025918,0.641137
8,0.001204,0.000281,0.964279,0.000632,0.000307,0.005768,0.000635,0.000361,0.001057,0.001355,0.005081,0.006183,0.002755,0.001753,0.002748,8.377334e-05,0.000575,0.000837,0.004103
9,0.071086,0.001465,0.001989,0.004688,0.000749,0.001275,0.034432,0.113764,0.018429,0.085285,0.011938,0.566021,0.010816,0.004681,0.000407,4.966744e-03,0.002335,0.012326,0.053348


In [13]:
kk = pd.read_csv('../pro/stacking_lr.csv')

In [17]:
for i,j in enumerate(name):
    if i == 0 or i==2 or i==1 or i==4 or i==6:
        continue
    else:
        kk[j] = df_lr[j]

In [16]:
kk

,class_prob_1,class_prob_2,class_prob_3,class_prob_4,class_prob_5,class_prob_6,class_prob_7,class_prob_8,class_prob_9,class_prob_10,class_prob_11,class_prob_12,class_prob_13,class_prob_14,class_prob_15,class_prob_16,class_prob_17,class_prob_18,class_prob_19
0,0.009009,0.002610,0.002393,0.004298,0.923713,0.005981,0.002343,0.012838,0.000302,0.006196,0.004514,0.008918,0.002520,0.003006,0.002101,0.003567,0.001142,0.001375,0.003177
1,0.008069,0.004334,0.001701,0.911992,0.002421,0.004217,0.012125,0.002047,0.000749,0.005303,0.004848,0.012742,0.006707,0.010269,0.003394,0.002164,0.001661,0.002878,0.002379
2,0.049700,0.008044,0.029827,0.009857,0.007118,0.014090,0.016473,0.071139,0.015731,0.017989,0.017516,0.078691,0.557428,0.014439,0.026310,0.007916,0.006476,0.010819,0.040436
3,0.006802,0.002982,0.002292,0.921651,0.003061,0.005720,0.006676,0.003331,0.000678,0.003681,0.006169,0.006391,0.004481,0.013684,0.002701,0.001704,0.002215,0.001604,0.004178
4,0.016428,0.002467,0.003703,0.003652,0.918365,0.002782,0.002143,0.008237,0.000398,0.012816,0.006157,0.006869,0.003177,0.002639,0.001078,0.003548,0.001019,0.001519,0.003002
5,0.016401,0.005300,0.002302,0.003172,0.882508,0.009153,0.001506,0.015054,0.000974,0.018996,0.009178,0.009721,0.003781,0.002222,0.002228,0.007749,0.000903,0.003592,0.005259
6,0.007733,0.001660,0.004222,0.001605,0.000356,0.002202,0.002015,0.003092,0.001190,0.005244,0.007000,0.004647,0.003488,0.002193,0.938903,0.000623,0.004403,0.002603,0.006820
7,0.032783,0.034648,0.012973,0.013020,0.004131,0.003266,0.031770,0.015968,0.008302,0.012356,0.013750,0.026385,0.030542,0.006285,0.009864,0.001986,0.053026,0.011494,0.677451
8,0.002040,0.000862,0.949886,0.000939,0.000924,0.004670,0.002636,0.000857,0.001419,0.002569,0.005820,0.008894,0.003933,0.001398,0.003732,0.000447,0.000927,0.001908,0.006139
9,0.052326,0.008349,0.003098,0.005589,0.005770,0.005557,0.021599,0.076849,0.006427,0.042398,0.020190,0.645901,0.024068,0.006123,0.004538,0.011096,0.006839,0.011577,0.041705


In [18]:
kk

,class_prob_1,class_prob_2,class_prob_3,class_prob_4,class_prob_5,class_prob_6,class_prob_7,class_prob_8,class_prob_9,class_prob_10,class_prob_11,class_prob_12,class_prob_13,class_prob_14,class_prob_15,class_prob_16,class_prob_17,class_prob_18,class_prob_19
0,0.009009,0.002610,0.002393,0.003967,0.923713,0.003302,0.002343,0.011183,0.000365,0.012444,0.004931,0.008785,0.003122,0.001779,0.001421,0.003912,0.000856,0.001809,0.002530
1,0.008069,0.004334,0.001701,0.917330,0.002421,0.004324,0.012125,0.002566,0.000754,0.005098,0.004763,0.010901,0.006155,0.011271,0.003289,0.002407,0.001617,0.002064,0.003770
2,0.049700,0.008044,0.029827,0.007601,0.007118,0.007139,0.016473,0.053339,0.005346,0.017416,0.013990,0.052323,0.697697,0.017912,0.010714,0.002494,0.006499,0.006715,0.024823
3,0.006802,0.002982,0.002292,0.919747,0.003061,0.004451,0.006676,0.002444,0.000658,0.005343,0.004534,0.009821,0.005605,0.011259,0.003300,0.002195,0.001600,0.001933,0.003769
4,0.016428,0.002467,0.003703,0.004055,0.918365,0.003489,0.002143,0.011348,0.000392,0.013320,0.004858,0.009037,0.003334,0.001780,0.001405,0.003912,0.000877,0.001678,0.002557
5,0.016401,0.005300,0.002302,0.004036,0.882508,0.003028,0.001506,0.013129,0.000398,0.016710,0.005641,0.008614,0.003070,0.001372,0.002085,0.005529,0.000738,0.001543,0.003092
6,0.007733,0.001660,0.004222,0.001144,0.000356,0.002527,0.002015,0.002859,0.000911,0.004991,0.007461,0.003885,0.003119,0.002225,0.943086,0.000472,0.003788,0.002230,0.005915
7,0.032783,0.034648,0.012973,0.008326,0.004131,0.005103,0.031770,0.012356,0.013220,0.006837,0.013261,0.032060,0.037627,0.010943,0.010610,0.001039,0.019469,0.025779,0.689424
8,0.002040,0.000862,0.949886,0.000901,0.000924,0.008903,0.002636,0.000717,0.001151,0.002219,0.006481,0.010380,0.003437,0.001303,0.003973,0.000527,0.000864,0.001711,0.005587
9,0.052326,0.008349,0.003098,0.005351,0.005770,0.004329,0.021599,0.047566,0.004995,0.019053,0.013326,0.715730,0.014805,0.009489,0.010384,0.012981,0.007343,0.004991,0.047423
